# 예제 3: 단일 경로 TOYO 데이터 분석

## 📁 분석 대상

Q7M Sub ATL 단일 테스트 데이터

**단일 폴더:**
- `Q7M Sub ATL [45v 2068mAh] [23] - 250219r`

## 🎯 분석 목표

- 단일 폴더 사이클 데이터 분석
- 용량, 효율, 온도, DCIR 변화 추적
- 특정 사이클 프로파일 분석 (충전/방전)

In [ ]:
# 환경 설정
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter, find_peaks

# 한글 폰트
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

# BatteryDataTool 모듈 Import
sys.path.append(r'c:\Users\Ryu\Python_project\data\battery251027')
from BatteryDataTool import (
    check_cycler, name_capacity, extract_text_in_brackets,
    toyo_cycle_data,
    toyo_chg_Profile_data, toyo_dchg_Profile_data
)

print("✅ 환경 설정 완료")

In [ ]:
# 데이터 경로 설정
BASE_PATH = r"C:\Users\Ryu\Python_project\data\BatteryData_251010\Rawdata"
FOLDER_NAME = "Q7M Sub ATL [45v 2068mAh] [23] - 250219r"
folder_path = os.path.join(BASE_PATH, FOLDER_NAME)

# 경로 확인
if os.path.exists(folder_path):
    print(f"✅ 폴더 경로 확인: {FOLDER_NAME}")
    
    # 충방전기 타입 확인
    is_toyo = check_cycler(folder_path) == False
    cycler_type = "TOYO" if is_toyo else "PNE"
    print(f"   충방전기: {cycler_type}")
    
    # 용량 추출 (대괄호 내)
    capacity_str = extract_text_in_brackets(FOLDER_NAME)
    if capacity_str and 'mAh' in capacity_str:
        capacity = int(''.join(filter(str.isdigit, capacity_str.split('mAh')[0])))
    else:
        capacity = name_capacity(FOLDER_NAME)
    
    print(f"   용량: {capacity} mAh")
    
    # 온도 추출
    temp_str = FOLDER_NAME.split('[')[-1].split(']')[0] if '[' in FOLDER_NAME else "23"
    print(f"   온도: {temp_str}°C")
else:
    print(f"❌ 폴더 경로 없음: {folder_path}")

In [ ]:
# 사이클 데이터 로딩
print("\n=== 사이클 데이터 로딩 중 ===\n")

if os.path.exists(folder_path):
    ini_crate = 0.5  # C-rate
    chkir = True
    
    try:
        cycle_df = toyo_cycle_data(folder_path, capacity, ini_crate, chkir)
        
        if cycle_df is not None and len(cycle_df) > 0:
            print(f"✅ 로딩 완료")
            print(f"   총 사이클 수: {len(cycle_df)}")
            print(f"   사이클 범위: {cycle_df.index.min()} ~ {cycle_df.index.max()}")
            print(f"\n   데이터 컬럼: {list(cycle_df.columns)}")
            print(f"\n데이터 미리보기:")
            print(cycle_df.head())
        else:
            print(f"⚠️ 데이터 없음")
            cycle_df = None
    
    except Exception as e:
        print(f"❌ 오류: {e}")
        cycle_df = None
else:
    print(f"❌ 경로 없음")
    cycle_df = None

In [ ]:
# 사이클 분석 그래프
if cycle_df is not None:
    fig, axes = plt.subplots(3, 2, figsize=(15, 12))
    fig.suptitle(f'Q7M Sub ATL 사이클 분석 ({capacity} mAh, {temp_str}°C)', 
                 fontsize=16, fontweight='bold')
    
    cycles = cycle_df.index.values
    
    # 1. 방전 용량
    axes[0, 0].plot(cycles, cycle_df['Dchg'], 'o-', color='#2E86AB', linewidth=2, markersize=4)
    axes[0, 0].set_xlabel('Cycle', fontsize=12)
    axes[0, 0].set_ylabel('방전 용량 비율', fontsize=12)
    axes[0, 0].set_title('1. 방전 용량', fontsize=12, fontweight='bold')
    axes[0, 0].set_ylim(0.70, 1.05)
    axes[0, 0].grid(True, alpha=0.3)
    axes[0, 0].axhline(y=0.8, color='r', linestyle='--', linewidth=1, alpha=0.5, label='80% EOL')
    axes[0, 0].legend()
    
    # 2. 충전 용량
    axes[0, 1].plot(cycles, cycle_df['Chg'], 'o-', color='#A23B72', linewidth=2, markersize=4)
    axes[0, 1].set_xlabel('Cycle', fontsize=12)
    axes[0, 1].set_ylabel('충전 용량 비율', fontsize=12)
    axes[0, 1].set_title('2. 충전 용량', fontsize=12, fontweight='bold')
    axes[0, 1].set_ylim(0.70, 1.05)
    axes[0, 1].grid(True, alpha=0.3)
    
    # 3. 충방 효율
    axes[1, 0].plot(cycles, cycle_df['Eff'], 'o-', color='#F18F01', linewidth=2, markersize=4)
    axes[1, 0].set_xlabel('Cycle', fontsize=12)
    axes[1, 0].set_ylabel('효율', fontsize=12)
    axes[1, 0].set_title('3. 충방 효율 (Dchg/Chg)', fontsize=12, fontweight='bold')
    axes[1, 0].set_ylim(0.95, 1.02)
    axes[1, 0].grid(True, alpha=0.3)
    
    # 4. 온도
    axes[1, 1].plot(cycles, cycle_df['Temp'], 'o-', color='#BC4B51', linewidth=2, markersize=4)
    axes[1, 1].set_xlabel('Cycle', fontsize=12)
    axes[1, 1].set_ylabel('온도 (°C)', fontsize=12)
    axes[1, 1].set_title('4. 온도 변화', fontsize=12, fontweight='bold')
    axes[1, 1].grid(True, alpha=0.3)
    
    # 5. DCIR
    if 'dcir' in cycle_df.columns:
        axes[2, 0].plot(cycles, cycle_df['dcir'], 'o-', color='#6A994E', linewidth=2, markersize=4)
        axes[2, 0].set_xlabel('Cycle', fontsize=12)
        axes[2, 0].set_ylabel('DCIR (mΩ)', fontsize=12)
        axes[2, 0].set_title('5. DCIR 변화', fontsize=12, fontweight='bold')
        axes[2, 0].grid(True, alpha=0.3)
    
    # 6. 용량-효율 상관관계
    axes[2, 1].scatter(cycle_df['Dchg'], cycle_df['Eff'], c=cycles, cmap='viridis', s=50, alpha=0.6)
    axes[2, 1].set_xlabel('방전 용량 비율', fontsize=12)
    axes[2, 1].set_ylabel('충방 효율', fontsize=12)
    axes[2, 1].set_title('6. 용량-효율 상관관계', fontsize=12, fontweight='bold')
    axes[2, 1].grid(True, alpha=0.3)
    cbar = plt.colorbar(axes[2, 1].collections[0], ax=axes[2, 1])
    cbar.set_label('Cycle', rotation=270, labelpad=15)
    
    plt.tight_layout()
    plt.show()

In [ ]:
# 통계 분석
if cycle_df is not None:
    print("\n" + "="*80)
    print("상세 통계 분석")
    print("="*80 + "\n")
    
    print(f"📊 기본 정보")
    print(f"   폴더: {FOLDER_NAME}")
    print(f"   용량: {capacity} mAh")
    print(f"   온도: {temp_str}°C")
    print(f"   총 사이클: {len(cycle_df)}")
    print(f"   사이클 범위: {cycle_df.index.min()} ~ {cycle_df.index.max()}")
    
    print(f"\n💾 방전 용량")
    initial_cap = cycle_df['Dchg'].iloc[0]
    final_cap = cycle_df['Dchg'].iloc[-1]
    print(f"   초기: {initial_cap:.4f} ({initial_cap * capacity:.1f} mAh)")
    print(f"   최종: {final_cap:.4f} ({final_cap * capacity:.1f} mAh)")
    print(f"   평균: {cycle_df['Dchg'].mean():.4f}")
    print(f"   최소: {cycle_df['Dchg'].min():.4f}")
    print(f"   최대: {cycle_df['Dchg'].max():.4f}")
    print(f"   용량 유지율: {(final_cap / initial_cap) * 100:.2f}%")
    
    # 80% EOL
    below_80 = cycle_df[cycle_df['Dchg'] < 0.8]
    if len(below_80) > 0:
        eol_cycle = below_80.index[0]
        print(f"   80% EOL 도달: Cycle {eol_cycle}")
    else:
        print(f"   80% EOL: 미도달")
    
    print(f"\n⚡ 효율")
    print(f"   평균: {cycle_df['Eff'].mean():.4f} ({cycle_df['Eff'].mean() * 100:.2f}%)")
    print(f"   최소: {cycle_df['Eff'].min():.4f}")
    print(f"   최대: {cycle_df['Eff'].max():.4f}")
    print(f"   표준편차: {cycle_df['Eff'].std():.4f}")
    
    print(f"\n🌡️ 온도")
    print(f"   평균: {cycle_df['Temp'].mean():.2f} °C")
    print(f"   최저: {cycle_df['Temp'].min():.2f} °C")
    print(f"   최고: {cycle_df['Temp'].max():.2f} °C")
    print(f"   표준편차: {cycle_df['Temp'].std():.2f} °C")
    
    if 'dcir' in cycle_df.columns:
        print(f"\n🔋 DCIR")
        print(f"   평균: {cycle_df['dcir'].mean():.2f} mΩ")
        print(f"   초기: {cycle_df['dcir'].iloc[0]:.2f} mΩ")
        print(f"   최종: {cycle_df['dcir'].iloc[-1]:.2f} mΩ")
        print(f"   증가율: {((cycle_df['dcir'].iloc[-1] / cycle_df['dcir'].iloc[0]) - 1) * 100:.2f}%")
    
    # 분위수 분석
    print(f"\n📈 분위수 분석 (방전 용량)")
    quantiles = cycle_df['Dchg'].quantile([0.25, 0.5, 0.75])
    print(f"   25%: {quantiles[0.25]:.4f}")
    print(f"   50% (중앙값): {quantiles[0.5]:.4f}")
    print(f"   75%: {quantiles[0.75]:.4f}")

In [ ]:
# 특정 사이클 프로파일 분석 (충전)
print("\n=== 충전 프로파일 분석 ===\n")

if cycle_df is not None and len(cycle_df) >= 10:
    # 분석할 사이클: 2, 10, 50 (또는 마지막 사이클)
    target_cycles = [2, 10, min(50, cycle_df.index.max())]
    
    try:
        profiles = []
        for cycle_num in target_cycles:
            if cycle_num in cycle_df.index:
                profile = toyo_chg_Profile_data(folder_path, cycle_num, capacity, 0.0, ini_crate, 51)
                if profile is not None and len(profile) > 0:
                    profiles.append({'cycle': cycle_num, 'data': profile})
                    print(f"✅ Cycle {cycle_num} 충전 프로파일 로딩 완료")
        
        if len(profiles) > 0:
            # 프로파일 그래프
            fig, axes = plt.subplots(1, 2, figsize=(15, 5))
            fig.suptitle('충전 프로파일 비교', fontsize=16, fontweight='bold')
            
            colors = ['#2E86AB', '#A23B72', '#F18F01']
            
            for i, item in enumerate(profiles):
                cycle_num = item['cycle']
                profile = item['data']
                color = colors[i % len(colors)]
                
                if 'V' in profile.columns and 'Time' in profile.columns:
                    # 전압 프로파일
                    axes[0].plot(profile['Time'], profile['V'], '-', color=color, 
                               linewidth=2, label=f'Cycle {cycle_num}')
                
                if 'I' in profile.columns and 'Time' in profile.columns:
                    # 전류 프로파일
                    axes[1].plot(profile['Time'], profile['I'], '-', color=color, 
                               linewidth=2, label=f'Cycle {cycle_num}')
            
            axes[0].set_xlabel('시간 (h)', fontsize=12)
            axes[0].set_ylabel('전압 (V)', fontsize=12)
            axes[0].set_title('전압 프로파일', fontsize=12, fontweight='bold')
            axes[0].grid(True, alpha=0.3)
            axes[0].legend()
            
            axes[1].set_xlabel('시간 (h)', fontsize=12)
            axes[1].set_ylabel('전류 (mA)', fontsize=12)
            axes[1].set_title('전류 프로파일', fontsize=12, fontweight='bold')
            axes[1].grid(True, alpha=0.3)
            axes[1].legend()
            
            plt.tight_layout()
            plt.show()
    
    except Exception as e:
        print(f"⚠️ 프로파일 로딩 실패: {e}")

In [ ]:
# Excel 저장 (옵션)
save_excel = True

if save_excel and cycle_df is not None:
    output_filename = f"{FOLDER_NAME.replace('[', '').replace(']', '').replace(' ', '_')}_분석.xlsx"
    output_path = os.path.join(BASE_PATH, output_filename)
    
    with pd.ExcelWriter(output_path, engine='xlsxwriter') as writer:
        # 사이클 데이터
        cycle_df.to_excel(writer, sheet_name='사이클_데이터')
        
        # 통계
        stats_data = {
            '항목': ['폴더명', '용량 (mAh)', '온도 (°C)', '총 사이클 수', 
                   '초기 용량 비율', '최종 용량 비율', '용량 유지율 (%)', 
                   '평균 효율', '평균 온도 (°C)'],
            '값': [
                FOLDER_NAME,
                capacity,
                temp_str,
                len(cycle_df),
                f"{cycle_df['Dchg'].iloc[0]:.4f}",
                f"{cycle_df['Dchg'].iloc[-1]:.4f}",
                f"{(cycle_df['Dchg'].iloc[-1] / cycle_df['Dchg'].iloc[0]) * 100:.2f}",
                f"{cycle_df['Eff'].mean():.4f}",
                f"{cycle_df['Temp'].mean():.2f}"
            ]
        }
        stats_df = pd.DataFrame(stats_data)
        stats_df.to_excel(writer, sheet_name='통계', index=False)
    
    print(f"\n✅ Excel 저장 완료: {output_path}")

---
## 📝 분석 결론

### 주요 발견사항

1. **사이클 특성**
   - 단일 폴더 데이터 완전 분석
   - 용량, 효율, 온도, DCIR 추세 파악

2. **성능 평가**
   - 초기 대비 최종 성능 비교
   - 80% EOL 도달 여부 확인

3. **프로파일 분석**
   - 특정 사이클의 충전 프로파일 비교
   - 시간에 따른 충전 특성 변화

---

**작성일**: 2025-10-28  
**데이터**: TOYO 단일 경로  
**배터리**: Q7M Sub ATL (2068 mAh, 23°C)